## Data introduction and import necessary Libraries

**Pixel Resolution:** 30m * 30m<br>
**Target:** Target data set<br>
**Point of Interest (POI):** POI Kernel Density, 
- POI_All, BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- POI_Sel, BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- POI_All: 1000m; POI_Sel: 500m<br>

**Road Network (RN):** Road Kernel Density
- BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- BandWidth: 1250m<br>

**NTL:** Time,2019/03; NPP-VIIRS, DNB<br>
**XM_Boundary:** mask of all the layers<br>
**Train_Test:** 
- Train, data located in Off-island area of Xiamen, label = 0
- Test,  data located in Island area of Xiamen, label = 1

In [1]:
import math
import json
import copy
import time
import numpy as np
import pandas as pd
from sl_1 import * # my custom module 

In [2]:
# Extract and generate headers
extrat_begin('Target', 'begin')

Generate header named begin.txt from layer Target


## Data preprocessing

### Read ASCII to array and generate header

In [3]:
def get_name(name_pref):
    '''
    input: name_pref, name prefixs of lyer name list
    output: a lyer name list
    '''
    ly_name = [name_pref + str(i) for i in range(250,2750,250)]
    return ly_name
# get_name('poi_all_')  # test example 1
# get_name('RN_') # test example 2
# get_name('poi_all_') + get_name('RN_') + ['Target','Train_Test','XM_Boundary']

In [4]:
# %%time
def read_ASC_Data(file_name, ly_names):
    '''input: file_name, a file name, parent file name of ly_names
       input: ly_names, a list, list of layer names in an ASCII data format
       output: ly_dict: a dictionary whose key is the layer name and the value is stored in an array format
    '''     
    ly_dict = {}
    for name in ly_names:
        ly_dict[name] = np.loadtxt('%s/\%s.txt' % (file_name, name), skiprows = 6)
    print('All ASCII data has been read')
    return ly_dict

### Data consistency test
Check whether there are missing values in different positions of the layer,<br>
and auto fill it if missing

In [5]:
def Na_Test(layer_names,stan_layer,ly_dict):
    '''input: layer_names, a list of layer names to be checked for missing values
       input: stan_layer, the normalized layer name used to verify that other layers have missing values
       ly_dict: the dict data read by read_ASC_Data function.
       output: For each layer in the name list, the following judgment is made. 
               If all the row raster cells of a layer are missing from the standard layer, 
               the layer is added to the result return list, 
               otherwise, the next layer is performed. Judgment.
    '''
    list1 = []
    for name in layer_names:
        # Fill in missing values start
        ly_dict[name][np.where(ly_dict[name] == -9999)] = 0 # Fill all missing values (- 9999) with 0
        ly_dict[name][np.where(ly_dict[stan_layer] == -9999)] = -9999 # Make - 9999 consistent with the standardized layer
        # Fill in missing values end
        a = max(ly_dict[name][np.where(ly_dict[stan_layer] == -9999)])
        b = min(ly_dict[name][np.where(ly_dict[stan_layer] != -9999)])
        if a == -9999 and b != -9999:
            continue
        else:
            list1.append(name)
    if len(list1) == 0:
        print('All layers pass the inspection, no missing values exist, and are consistent with the standardized layer')
    else:
        print('The following layers have missing values')
        print(list1)
    return
### test
# test_name = get_name('poi_all_')
# ly_dict = copy.deepcopy(read_ASC_Data('ASCII', test_name))
# Na_Test(test_name[0],'XM_Boundary',ly_dict)

### Dimension reduction

In [6]:
def revel_array(dict_one):
    '''input: dict_one, a dictionary whose key is the layer name and the value is stored in an array format
       output: A dict whose value is all converted to one-dimensional array
    '''
    new_dict = copy.deepcopy(dict_one)
    for key,value in new_dict.items():
        new_dict[key] = np.ravel(value, order='C') # Expand to one dimension by row
    num_rows = len(list(new_dict.values())[0])
    new_dict['ID'] = np.array([i for i in range(num_rows)]) # Build index with name ID
    return new_dict

# ly_dict_reval = revel_array(ly_dict)
# ly_dict_reval
# len(ly_dict_reval['Target']) # show total number of records

### conversion to dataframe and to csv

In [7]:
def ly_df_csv(input_file_loc,ly_names,write_name):
    '''
    input: input_file_loc, file loction name of input data
    input: ly_names, , a list of layer names to be combined
    output: a csv file named write_name
    '''
    ly_dict0 = read_ASC_Data(input_file_loc, ly_names) # read ASCII data
    ly_dict = copy.deepcopy(ly_dict0)
#     print(ly_dict['Target'])
#     print(np.shape(ly_dict['Target']))
    test_name = ly_names
    Na_Test(test_name[:-1],test_name[-1], ly_dict) # test_name[-1], stan_layer, in this case is 'XM_Boundary'
    ly_dict_reval = revel_array(ly_dict) # Dimension reduction
#     print(len(ly_dict_reval['Target'])) # show total number of records
    ly_df = pd.DataFrame(ly_dict_reval,columns=ly_dict_reval.keys())
    ly_df.to_csv(r'data\%s.csv' % (write_name),index = False)
    print('Data writing is over')
    return

### write three csv data

In [8]:
input_file_loc = 'ASCII'
ly_names = get_name('poi_all_') + get_name('poi_sel_') + get_name('RN_') + ['NTL','Target','Train_Test','XM_Boundary']
write_name = 'ly_df'
ly_df_csv(input_file_loc,ly_names,write_name) # write the csv file

All ASCII data has been read
All layers pass the inspection, no missing values exist, and are consistent with the standardized layer
Data writing is over


### Data cleaning

#### Remove missing values

In [21]:
data = pd.read_csv("data/ly_df.csv")

In [22]:
# data = ly_df.copy()
print(len(data))
data.columns

3450552


Index(['poi_all_250', 'poi_all_500', 'poi_all_750', 'poi_all_1000',
       'poi_all_1250', 'poi_all_1500', 'poi_all_1750', 'poi_all_2000',
       'poi_all_2250', 'poi_all_2500', 'poi_sel_250', 'poi_sel_500',
       'poi_sel_750', 'poi_sel_1000', 'poi_sel_1250', 'poi_sel_1500',
       'poi_sel_1750', 'poi_sel_2000', 'poi_sel_2250', 'poi_sel_2500',
       'RN_250', 'RN_500', 'RN_750', 'RN_1000', 'RN_1250', 'RN_1500',
       'RN_1750', 'RN_2000', 'RN_2250', 'RN_2500', 'Target', 'Train_Test',
       'XM_Boundary', 'ID', 'NTL'],
      dtype='object')

In [23]:
clean_data = data.copy()
clean_data = clean_data[clean_data.loc[:,'Target'] != -9999]
clean_data.head()

,poi_all_250,poi_all_500,poi_all_750,poi_all_1000,poi_all_1250,poi_all_1500,poi_all_1750,poi_all_2000,poi_all_2250,poi_all_2500,...,RN_1500,RN_1750,RN_2000,RN_2250,RN_2500,Target,Train_Test,XM_Boundary,ID,NTL
430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002976,0.020894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,430,-9999.0
431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001634,0.018002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,431,-9999.0
2361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005609,0.025637,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2361,-9999.0
2362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003711,0.022329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2362,-9999.0
2363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002184,0.019295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2363,-9999.0


In [24]:
len(clean_data)

1890571

#### add lng and lat of each pixel

In [25]:
coor = np.loadtxt('data/coor.txt') # load coordinate data
coor[0:6]

array([[118.011595,  24.907131],
       [118.011892,  24.907129],
       [118.011296,  24.906862],
       [118.011593,  24.90686 ],
       [118.01189 ,  24.906858],
       [118.012187,  24.906856]])

In [27]:
len(coor)

1890571

In [28]:
clean_data['lng'] = coor[:,0]
clean_data['lat'] = coor[:,1]

In [29]:
clean_data.columns

Index(['poi_all_250', 'poi_all_500', 'poi_all_750', 'poi_all_1000',
       'poi_all_1250', 'poi_all_1500', 'poi_all_1750', 'poi_all_2000',
       'poi_all_2250', 'poi_all_2500', 'poi_sel_250', 'poi_sel_500',
       'poi_sel_750', 'poi_sel_1000', 'poi_sel_1250', 'poi_sel_1500',
       'poi_sel_1750', 'poi_sel_2000', 'poi_sel_2250', 'poi_sel_2500',
       'RN_250', 'RN_500', 'RN_750', 'RN_1000', 'RN_1250', 'RN_1500',
       'RN_1750', 'RN_2000', 'RN_2250', 'RN_2500', 'Target', 'Train_Test',
       'XM_Boundary', 'ID', 'NTL', 'lng', 'lat'],
      dtype='object')

#### Reorder and save clean data

In [30]:
order = ['poi_all_250', 'poi_all_500', 'poi_all_750', 'poi_all_1000',
       'poi_all_1250', 'poi_all_1500', 'poi_all_1750', 'poi_all_2000',
       'poi_all_2250', 'poi_all_2500', 'poi_sel_250', 'poi_sel_500',
       'poi_sel_750', 'poi_sel_1000', 'poi_sel_1250', 'poi_sel_1500',
       'poi_sel_1750', 'poi_sel_2000', 'poi_sel_2250', 'poi_sel_2500',
       'RN_250', 'RN_500', 'RN_750', 'RN_1000', 'RN_1250', 'RN_1500',
       'RN_1750', 'RN_2000', 'RN_2250', 'RN_2500', 'NTL',
       'lng', 'lat', 'Target', 'Train_Test', 'XM_Boundary', 'ID']
clean_data = clean_data[order]
# clean_data.rename(columns={'POI_Sel':'POI'}, inplace = True)
clean_data.to_csv(r'data\ly_df_clean.csv',index = False)

#### read clean data and split it

In [2]:
clean_data = pd.read_csv(r'data\ly_df_clean.csv')
clean_data.head()

,poi_all_250,poi_all_500,poi_all_750,poi_all_1000,poi_all_1250,poi_all_1500,poi_all_1750,poi_all_2000,poi_all_2250,poi_all_2500,...,RN_2000,RN_2250,RN_2500,NTL,lng,lat,Target,Train_Test,XM_Boundary,ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002976,0.020894,...,0.0,0.0,0.0,-9999.0,118.011595,24.907131,0.0,0.0,1.0,430
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001634,0.018002,...,0.0,0.0,0.0,-9999.0,118.011892,24.907129,0.0,0.0,1.0,431
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005609,0.025637,...,0.0,0.0,0.0,-9999.0,118.011296,24.906862,0.0,0.0,1.0,2361
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003711,0.022329,...,0.0,0.0,0.0,-9999.0,118.011593,24.906860,0.0,0.0,1.0,2362
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002184,0.019295,...,0.0,0.0,0.0,-9999.0,118.011890,24.906858,0.0,0.0,1.0,2363


In [ ]:
len(clean_data) # Sample size: 1890571

### spli train and test
- Train, data located in Off-island area of Xiamen, label = 0
- Test,  data located in Island area of Xiamen, label = 1

In [8]:
# Train, Test = train_test_split(clean_data, test_size=0.33, random_state=160) 
# orignal split but don't have spatial independence

In [3]:
Train = clean_data[clean_data['Train_Test'] == 0]
Test = clean_data[clean_data['Train_Test'] == 1]
# Train.to_csv(r'data\Train.csv')
# Test.to_csv(r'data\Test.csv')
# Train.head()
# Test.head()

## Models

In [4]:
data1 = pd.read_csv("data/POI_Sel.csv")

In [5]:
a1 = data1['adname'].isin(['集美区','同安区','翔安区']) # Off-island
a2 = data1['adname'].isin(['思明区','湖里区']) # in island
Train_POI = data1.loc[a1]
Test_POI = data1.loc[a2]

### SVM

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn import svm
X = np.array(Train['poi_all_250']).reshape(-1, 1)
y = Train['Target']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y,
                        scoring='precision_macro', cv=3,
                        return_estimator=True)
sorted(scores.keys())


In [4]:
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
X = np.array(Train['poi_all_250']).reshape(-1, 1)
y = Train['Target']
lasso = linear_model.Lasso()

cv_results = cross_validate(lasso, X, y, cv=3, return_train_score=True)
# sorted(cv_results.keys())
# ['fit_time', 'score_time', 'test_score', 'train_score']
cv_results['test_score']

### KNN

### Decision tree

In [71]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

def get_name(name_pref):
    '''
    input: name_pref, name prefixs of lyer name list
    output: a lyer name list
    '''
    ly_name = [name_pref + str(i) for i in range(250,2750,250)]
    return ly_name
# get_name('poi_all_')  # test example 1
# get_name('RN_') # test example 2
# get_name('poi_all_') + get_name('RN_') + ['Target','Train_Test','XM_Boundary']

def opti_band(names):
    '''input: names, a list made up of different band width name
       output: a string, a band width name with max score
    '''
    score_list = []
    for name in names:
        X = np.array(Train[name]).reshape(-1, 1)
        y = Train['Target']
        clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
        scores = cross_val_score(clf, X, y, cv=5)
        score_list.append(scores.mean())
    return score_list

def select_name(names):
    A = {}
    scores = opti_band(names)
    ind = scores.index(max(scores))
    name = names[ind]
    A["name"] = name
    A["scores"] = scores
    return A
# input_name = 'poi_all_' # 
# names = get_name(input_name)[0:2]
# name = select_name(names)
# name

In [72]:
for input_name in ['poi_sel_250', 'RN_500']:
    names = get_name(input_name)[0:2]
    name = select_name(names)
    name

{'name': 'poi_all_250', 'scores': [0.837207688762503, 0.8355518447889045]}

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
for name in ['poi_sel_250', 'RN_500','NTL']:
    X_Train =  np.array(Train[name]).reshape(-1, 1)
    Y_Train = Train['Target'].copy()
    X_Test = np.array(Test[name]).reshape(-1, 1)
    Y_Test = Test['Target'].copy() # get sub dataframe
    clf = DecisionTreeClassifier(max_leaf_nodes=2, random_state=1)
    clf.fit(X_Train, Y_Train)
    predictions = clf.predict(X_Test)
    acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
    print(acc)

0.8591486166330017
0.8435773701765518
0.8834397611050637


In [69]:
from sklearn.model_selection import GridSearchCV
X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
Y_Train = Train['Target'].copy()
params = {'max_leaf_nodes': list(range(2, 10))}
grid = GridSearchCV(DecisionTreeClassifier(), params, cv=5, iid=False)
grid.fit(X_Train, Y_Train)
print("leaf_side: {0}".format(grid.best_estimator_.max_leaf_nodes))
# use the best estimator to compute the kernel density estimate
clf = grid.best_estimator_

leaf_side: 4


In [70]:
# X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
# Y_Train = Train['Target'].copy()
# X_Test = np.array(Test[['poi_sel_250', 'RN_500','NTL']])
# Y_Test = Test['Target'].copy() # get sub dataframe
# clf = DecisionTreeClassifier(max_leaf_nodes=4, random_state=1)
# clf.fit(X_Train, Y_Train)
predictions = clf.predict(X_Test)
acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
print(acc)

0.8479023930240815


## Ensemble methods
- RandomForest
- AdaBoost
- Gradient Tree Boosting
- Voting Classifier

In [13]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

names = ["LogisticRegression",
         "Decision Tree", 
         "Random Forest", 
         "Neural Net", 
         "AdaBoost",
         "Naive Bayes", 
         "QDA"]

classifiers = [
    LogisticRegression(penalty='l2', solver='sag',max_iter =150),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=50),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [14]:
from sklearn.metrics import f1_score

X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
Y_Train = Train['Target'].copy()
X_Test = np.array(Test[['poi_sel_250', 'RN_500','NTL']])
Y_Test = Test['Target'].copy() # get sub dataframe


for name, clf in list(zip(names, classifiers))[1:2]:
    clf.fit(X_Train, Y_Train)
    score = clf.score(X_Test, Y_Test) # overall_accuracy
    predictions = clf.predict(X_Test)
    f1 = f1_score(Y_Test, predictions, average='macro') # f1_score
    print('classifier', name, "overall_accuracy:", score,'f1_score',f1)

classifier LogisticRegression overall_accuracy: 0.8391766739817569 f1_score 0.8082362025798979


D:\Program\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [40]:
2:3

0.8709885849666165

In [ ]:
0.1 0.01 0.001 1e-4 1e-5

In [11]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

In [ ]:
estimators = [("RandomForest", RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)),
              ("AdaBoost", AdaBoostClassifier(n_estimators=100)),
              ("Gradient Tree Boosting", GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0))
             ]

### Random Forest

In [73]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def get_name(name_pref):
    '''
    input: name_pref, name prefixs of lyer name list
    output: a lyer name list
    '''
    ly_name = [name_pref + str(i) for i in range(250,2750,250)]
    return ly_name
# get_name('poi_all_')  # test example 1
# get_name('RN_') # test example 2
# get_name('poi_all_') + get_name('RN_') + ['Target','Train_Test','XM_Boundary']

def opti_band(names):
    '''input: names, a list made up of different band width name
       output: a string, a band width name with max score
    '''
    score_list = []
    for name in names:
        X = np.array(Train[name]).reshape(-1, 1)
        y = Train['Target']
        clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
        scores = cross_val_score(clf, X, y, cv=5)
        score_list.append(scores.mean())
    return score_list

def select_name(names):
    A = {}
    scores = opti_band(names)
    ind = scores.index(max(scores))
    name = names[ind]
    A["name"] = name
    A["scores"] = scores
    return A
# input_name = 'poi_all_' # 
# names = get_name(input_name)[0:2]
# name = select_name(names)
# name

In [74]:
input_name = 'poi_sel_'
names = get_name(input_name)[0:2]
name = select_name(names)
name

{'name': 'poi_sel_500', 'scores': [0.8580399328355434, 0.8602147154954132]}

In [12]:
X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
Y_Train = Train['Target'].copy()
X_Test = np.array(Test[['poi_sel_250', 'RN_500','NTL']])
Y_Test = Test['Target'].copy() # get sub dataframe
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

clf.fit(X_Train, Y_Train)
predictions = clf.predict(X_Test)
acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
print(acc)

0.8428963100511668


In [76]:
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

### AdaBoost

In [77]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
Y_Train = Train['Target'].copy()
X_Test = np.array(Test[['poi_sel_250', 'RN_500','NTL']])
Y_Test = Test['Target'].copy() # get sub dataframe

clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_Train, Y_Train)
predictions = clf.predict(X_Test)
acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
print(acc)


0.8917987554644888


### Gradient Tree Boosting

In [78]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

X_Train =  np.array(Train[['poi_sel_250', 'RN_500','NTL']])
Y_Train = Train['Target'].copy()
X_Test = np.array(Test[['poi_sel_250', 'RN_500','NTL']])
Y_Test = Test['Target'].copy() # get sub dataframe

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)

clf.fit(X_Train, Y_Train)
predictions = clf.predict(X_Test)
acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
print(acc)

0.8896042283938042


## K-fold cross validation3

input_file_loc = 'ASCII'
ly_name = 'Train_Test'
read_ASC_Data(file_name, ly_names):

## Classification of Urban Built-up Areas

### import sklearn libraries and read data

import math
import json
import copy
import time
import numpy as np
import pandas as pd
from sl_1 import * # my custom module 
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,precision_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

ly_df = pd.read_csv(r'data\ly_df.csv')
ly_df.head()

### change number of nodes

In [9]:
from itertools import combinations
def get_comb(input_list,n):
    '''input: input_list, a list.
       input: n，an integer; the number of combinations.
       output: a list, the combinations of input_list.
    '''
    comb_all = []
    m = 1
    while m<n+1:
        a = [list(i) for i in list(combinations(input_list, m))]
        comb_all += a
        m += 1
    return comb_all
comb_all = get_comb([1,2,3], 3)
comb_all

[[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]

In [11]:
# %%time : Query time, but the variable is not saved

def get_acc(ind,node_num):
#     x_features = clean_data.iloc[:,1:ind].columns
    x_features = clean_data.iloc[:,ind].columns
    y_col = 'Target'
    X_Train = Train[x_features].copy()
    X_Test = Test[x_features].copy()
    Y_Train = Train[[y_col]].copy()
    Y_Test = Test[[y_col]].copy() # get sub dataframe
    clf = DecisionTreeClassifier(max_leaf_nodes=node_num, random_state=1)
    clf.fit(X_Train, Y_Train)
    predictions = clf.predict(X_Test)
#     acc = accuracy_score(y_true = Y_Test, y_pred = predictions) # use f1-score instead
    tn, fp, fn, tp = confusion_matrix(y_true = Y_Test, y_pred = predictions).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2*precision*recall/(precision+recall)
    return f1_score
# Permutations
# l = [[get_acc(i,j) for i in get_comb([1,2,3], 3)] for j in range(2,51)]
def acc_node(a,b): # a=2,b=51
    dict1 = {}
    for col_inx in get_comb([1,2,3], 3):
        list1 = []
        for j in range(a,b):
            list1.append(get_acc(col_inx,j))
            if len(col_inx) <= 1:
                ASC_Name = clean_data.columns[col_inx][0]
            else:
                ASC_Name = '_'.join(clean_data.columns[col_inx])
        dict1[ASC_Name] = list1
    return dict1


In [12]:
import time
t1 = time.time()
DT_node = acc_node(2,31)
t2 = time.time()
print('finised..%s min' % (round((t2-t1)/60,2)))
# time: I use 9.8 min 

DT_nodes = pd.DataFrame(DT_node,index =list(range(2,31)))
DT_nodes.to_csv(r'data\DT_node.csv',index_label = 'nodes')
# DT_nodes = pd.read_csv(r'data\DT_node.csv')

## prediction

In [12]:
def function(a, b): # a: true target; b: predict
    arr1 = np.zeros(shape=(len(a),1))
    arr1[(b == 0) & (a == 0)] = 0 # TN
    arr1[(b == 1) & (a == 1)] = 1 # TP
    arr1[(b == 1) & (a == 0)] = 2 # FP
    arr1[(b == 0) & (a == 1)] = 3 # FN
    return arr1
# a = np.array([0,0,1,1])
# b = np.array([0,1,1,0])
# function(a, b)
def para_dict(clean_data, col_inx, node_num):
    '''input: clean_data, a cleaned dataset (no NA) used to predict the built area.
       input: col_inx, a columns index list of cleaned data used to predict urban built.
       input: node_num, a number of nodes of decison tree.
       output: the predicted result and parameter of decision tree.
    '''
    dict0 = {}
    x_features = clean_data.iloc[:,col_inx].columns
    y_col = 'Target'
    X_Train = Train[x_features].copy()
    X_Test = Test[x_features].copy()
    Y_Train = Train[[y_col]].copy()
    Y_Test = Test[[y_col]].copy() # get sub dataframe
    clf = DecisionTreeClassifier(max_leaf_nodes=node_num, random_state=1)
    clf.fit(X_Train, Y_Train)
    predictions = clf.predict(X_Test)
# test in test data
#     acc1 = accuracy_score(y_true = Y_Test, y_pred = predictions) # accuracy of test data
    
    tn, fp, fn, tp = confusion_matrix(y_true = Y_Test, y_pred = predictions).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2*precision*recall/(precision+recall)
    proba = clf.predict_proba(X_Test)
    log_proba = clf.predict_log_proba(X_Test)

# prediction
    x_data = clean_data[x_features].copy()
    y_data = clean_data[[y_col]].copy()
    pred_y = clf.predict(x_data)
# over_all accuracy
#     acc2 = accuracy_score(y_true = y_data, y_pred = pred_y)
#     overall_kappa = cohen_kappa_score(y_data, pred_y)
#     proba = clf.predict_proba(x_data)
#     log_proba = clf.predict_log_proba(x_data)

    data1 = clean_data.copy()
    ins_loc = len(clean_data.columns) # insert it to end
    data1.insert(ins_loc,'pred_y',pred_y)
    a = np.array(data1.Target)
    b = pred_y
    data1['pred_y1'] = function(a, b)
    
    if len(col_inx) <= 1:
        ASC_Name = clean_data.columns[col_inx][0]
    else:
        ASC_Name = '_'.join(clean_data.columns[col_inx])
### add to dictionary
    dict0['Name'] =  ASC_Name
    dict0['Precision'] = round(precision,6) # pre
    dict0['F1_score'] = round(f1_score,6) # pre
    dict0['Proba'] = proba
    dict0['Log_Proba'] = log_proba
    dict0['Recall'] = round(recall,6)
    dict0['Pred_y'] = pred_y
    dict0['Pred_y1'] = np.array(data1['pred_y1'])
#     dict0['Acc_Test'] = round(acc1,6)
#     dict0['Acc_All'] = round(acc2,6)
#     dict0['Kappa'] = round(overall_kappa,6)
#     dict0['Target'] = np.array(y_data).ravel() # overall
    dict0['Target'] = np.array(Y_Test).ravel()
    dict0['TN'] = tn
    dict0['FP'] = fp
    dict0['FN'] = fn
    dict0['TP'] = tp
    return dict0
# %%time
# 5.93 s
# para_0 = para_dict(clean_data, [1,2], 11)
# para_0

In [13]:
# %%time
# Wall time: 27.1 s
def para_dict_all(data, col_inx_list, node_num):
    '''input: col_inx, a list, a columns index list of cleaned data used to predict urban built.
       input: node_num, a number of nodes of decison tree.
       output: the predicted result and parameter of decision tree.
    '''
    dict1 = {}
    i = 0
    for col_inx in col_inx_list:
        para_0 = para_dict(data, col_inx, node_num)
        dict1[str(i)] = para_0
        i += 1
    return dict1
# para_DT = para_dict_all(clean_data, [[1],[2],[3],[1,2,3]], 11)
# para_DT

In [14]:
# %%time
# Wall time: 17.9 s
def pred_ASC(raw_data, clean_data, para_all, w_name, Bool=1):
    '''input:raw_data, a raw dataset (have NA) used to predict the built area.
       input:clean_data, a cleaned dataset (no NA) used to predict the built area.
       input:para_all, a dict, the predicted result and parameter of decision tree.
       input:w_name, the output file folder name.
       input:Bool, a bool value of '0' or '1', defult is 1; '0' means pred y have 
             value(0 and 1) while '1' means pred y have  value(0, 1, 2 and 3).
       output: print the pred_y to a ASCII file.
    '''
    print('The program starts running')
    for key,value in para_all.items():
        if key != 'Target':
            name = value['Name']
            if Bool == 1:
                pred_y = value['Pred_y1']
            elif Bool == 0:
                pred_y = value['Pred_y']
            else:
                print('unexpected input variable of Bool. Bool are suposed to be a bool value of "0" or "1" ')
            data1 = clean_data.copy()
            data1['pred_y'] = pred_y
            data2 = pd.merge(raw_data, data1.iloc[:,[-2,-1]], how='left', left_on='ID', right_on='ID', sort=True)
            data3 = data2.copy().fillna(-9999)
            shape_ras = (1786, 1932) # Number of rows and columns
            built_pre = np.array(data3['pred_y']).reshape(shape_ras)
            np.savetxt('data/%s/%s.txt' % (w_name,name), built_pre, fmt='%0.0f') # Skip the first 6 lines
            add_begin('%s/%s' % (w_name,name))
        else:
            pass
    print('The program has finished running')
# para_DT = para_dict_all(clean_data, [[1],[2],[3],[1,2,3]], 11)
# pred_ASC(raw_data, clean_data, para_DT, pred_DT,Bool = 1)

In [15]:
raw_data = pd.read_csv(r'data\ly_df.csv')
# raw_data.head()
clean_data = pd.read_csv(r'data\ly_df_clean.csv')
clean_data.head()

,Target,RN,NTL,POI,XM_Boundary,ID
0,0.0,0.0,0.0,0.0,1.0,430
1,0.0,0.0,0.0,0.0,1.0,431
2,0.0,0.0,0.0,0.0,1.0,2361
3,0.0,0.0,0.0,0.0,1.0,2362
4,0.0,0.0,0.0,0.0,1.0,2363


In [16]:
Train, Test = train_test_split(clean_data, test_size=0.33, random_state=160)

## Save predicted data to ASCII

### to ASCII
- NTL.txt
- POI.txt
- RN.txt
- RN_NTL_POI.txt

In [18]:
para_DT = para_dict_all(clean_data, [[1],[2],[3],[1,2,3]], 11)
pred_ASC(raw_data, clean_data, para_DT, 'pred_DT',Bool = 1)

The program starts running
The program has finished running


### save parameter to csv
- precision
- recall
- f1-score

In [25]:
para_DT['0']

In [27]:
df_para = pd.DataFrame(para_DT).T
# df_para.head()
# order = ['Name','Acc_Test','Acc_All','Kappa','TN','TP','FN','FP','Precision','Recall']
order = ['Name','TN','TP','FN','FP','Precision','Recall','F1_score']
df_para = df_para[order]
df_para.to_csv('data/pred_DT/para_DT.csv', index = False)
# new_para = pd.read_csv('data/pred_DT/para_DT.csv')